In [8]:
!pip install --upgrade tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 41kB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 3.8MB 49.6MB/s 
     |████████████████████████████████| 512kB 50.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=c82aa7b364bccede30f456976d1d50834ef820b1f9d2ae60dd8a97c037530563
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [4]:
# 問題1 ディープラーニングに必要な内容を列挙する

# エポック、イテレーション、バッチサイズ
# 重みとバイアス
# 重みとバイアスの初期化
# 活性化関数
# forward
# アルゴリズム
# loss
# backward

In [12]:
import pandas as pd
import tensorflow as tf

iris = pd.read_csv("Iris.csv")
iris[iris["Species"] == "Iris-versicolor"]
iris[iris["Species"] == "Iris-virginica"]

iris2 = iris["Species"][50:]
iris2

50     Iris-versicolor
51     Iris-versicolor
52     Iris-versicolor
53     Iris-versicolor
54     Iris-versicolor
            ...       
145     Iris-virginica
146     Iris-virginica
147     Iris-virginica
148     Iris-virginica
149     Iris-virginica
Name: Species, Length: 100, dtype: object

In [122]:
# 問題2
#　下記コードをみてディープラーニングを実装するために必要なもの」が
# TensorFlowではどう実装されているかを確認してください
# 考察は下記に記載してきます。 

"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
# データセットの読み込み
df = pd.read_csv("Iris.csv")
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor") | (df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
# NumPy 配列に変換
X = np.array(X)
y = np.array(y)
# ラベルを数値に変換
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y = y.astype(np.int64)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    tf.random.set_random_seed(0)
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
# tf.nn.sigmoid_cross_entropy_with_logits：クロスエントロピーの値
# Y = (n, n_class)
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op) #?????
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))
  
# エポック、イテレーション、バッチサイズ
# ⇨エポック： for epoch in range(num_epochs):
# ⇨イテレーション：get_mini_batch_train
# ⇨バッチサイズ：total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
# ほぼNNと同じ

# 重みとバイアス
# ⇨重み　:　tf.Variable(tf.random_normal([n_input, n_hidden1]))
# ⇨バイアス：tf.Variable(tf.random_normal([n_hidden1]))
# NNとは違いtf.Variableを使っているが、基本は同じ。
# numpyでもnp.random.randn(n_input, n_hidden1)と同じ。

# 重みとバイアスの初期化
# ⇨初期化：tf.global_variables_initializer()
# NNでは最初に上記のnp.random.randn(n_input, n_hidden1)で求める
# テンソルでは短くて便利

# 活性化関数
# ⇨tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# クロスエントロピーとかぶっているがほとんど同じ。

# forward
# ⇨sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
# NNと比べて非常に楽。
# train_opがAdamなので勾配計算のアルゴリズムをエポック分回せば学習が可能な点が非常に強い

# アルゴリズム
# ⇨optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
# NNとほとんど差はない。

# loss
# ⇨oss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# lossでは、基本y_pred/y_trueの式だがtfでは平均でlossを取っている

# backward !!!!!!!!!!!!!!!!!!!!!!!!!!!▽▽黒●
# ⇨train_op = optimizer.minimize(loss_op) 
# バックワードもlossを入れて簡単に処理している

# 全体的な意見
# スクラッチと比べコード量が大幅に少なくなって、とても使いやすい

Epoch 0, loss : 6.0437, val_loss : 25.4480, acc : 0.625
Epoch 1, loss : 2.4252, val_loss : 5.6690, acc : 0.312
Epoch 2, loss : 1.2547, val_loss : 17.3184, acc : 0.375
Epoch 3, loss : 1.2396, val_loss : 10.4903, acc : 0.375
Epoch 4, loss : 0.5408, val_loss : 2.5693, acc : 0.562
Epoch 5, loss : 0.4709, val_loss : 2.3007, acc : 0.438
Epoch 6, loss : 0.3533, val_loss : 2.3549, acc : 0.438
Epoch 7, loss : 0.1935, val_loss : 0.4953, acc : 0.750
Epoch 8, loss : 0.1442, val_loss : 1.0494, acc : 0.562
Epoch 9, loss : 0.0750, val_loss : 0.1496, acc : 0.938
Epoch 10, loss : 0.0582, val_loss : 0.3923, acc : 0.812
Epoch 11, loss : 0.0349, val_loss : 0.0535, acc : 1.000
Epoch 12, loss : 0.0318, val_loss : 0.0843, acc : 1.000
Epoch 13, loss : 0.0249, val_loss : 0.0468, acc : 1.000
Epoch 14, loss : 0.0197, val_loss : 0.0371, acc : 1.000
Epoch 15, loss : 0.0174, val_loss : 0.0413, acc : 1.000
Epoch 16, loss : 0.0140, val_loss : 0.0388, acc : 1.000
Epoch 17, loss : 0.0120, val_loss : 0.0427, acc : 1.000

In [123]:
# 問題3
# ワンほっとを使っていく
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

y = iris.loc[:,"Species"]
X = iris.iloc[:, 1:5]

# NumPy 配列に変換
X = np.array(X)
y = np.array(y)

# ラベルを数値に変換
y[y == "Iris-setosa"] = 0 
y[y == "Iris-versicolor"] = 1 
y[y == "Iris-virginica"] = 2 

# one-hotのためにreshape(-1,1)
y = y.reshape(-1,1)

# hot
scaler = OneHotEncoder()
y_hot = scaler.fit_transform(y).toarray()
print(y_hot)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0.

In [124]:
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X,y_hot, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [125]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    tf.random.set_random_seed(0)
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
# tf.nn.sigmoid_cross_entropy_with_logits：クロスエントロピーの値
# Y = (n, n_class)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
a = tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits)
#b = tf.argmax(tf.nn.softmax(logits), axis=1)

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op) 
# 推定結果
correct_pred = tf.equal(tf.sign(tf.argmax(Y, axis=1) - 1), tf.sign(tf.argmax(tf.nn.softmax(logits), axis=1) - 1))


# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            # print(mini_batch_y)
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            #aq = sess.run(a, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            #bq = sess.run(b, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            #yq = sess.run(Y, feed_dict={Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        #print(aq)
        #print(aq.shape)
        #print(yq)
        #print(bq)
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

# 考察
# 変更点は２箇所
# f.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
# tf.sign(tf.argmax(Y, axis=1) - 1), tf.sign(tf.argmax(tf.nn.softmax(logits), axis=1) - 1))
# 2値分類から多クラス分類が必要なので、多クラス分類を探した結果、ソフトマックス関数とセットのものを発見（厳密にはsigmoidではセットで作られた関数がなかった）
# 多クラスになったので、argmaxで最大のインデックスを所得して差し引くことで整合を確認


Epoch 0, loss : 29.8890, val_loss : 268.8247, acc : 0.292
Epoch 1, loss : 22.0287, val_loss : 189.8217, acc : 0.292
Epoch 2, loss : 14.4276, val_loss : 112.7800, acc : 0.292
Epoch 3, loss : 7.0381, val_loss : 40.8063, acc : 0.458
Epoch 4, loss : 2.6262, val_loss : 27.7799, acc : 0.375
Epoch 5, loss : 0.9484, val_loss : 13.8942, acc : 0.708
Epoch 6, loss : 0.4914, val_loss : 6.8456, acc : 0.708
Epoch 7, loss : 0.1057, val_loss : 3.5524, acc : 0.792
Epoch 8, loss : 0.0637, val_loss : 2.8730, acc : 0.833
Epoch 9, loss : 0.0694, val_loss : 2.9574, acc : 0.833
Epoch 10, loss : 0.0650, val_loss : 3.2462, acc : 0.875
Epoch 11, loss : 0.0616, val_loss : 3.2697, acc : 0.875
Epoch 12, loss : 0.0594, val_loss : 3.1311, acc : 0.875
Epoch 13, loss : 0.0586, val_loss : 3.0505, acc : 0.875
Epoch 14, loss : 0.0579, val_loss : 3.0476, acc : 0.875
Epoch 15, loss : 0.0567, val_loss : 3.0499, acc : 0.875
Epoch 16, loss : 0.0555, val_loss : 3.0033, acc : 0.875
Epoch 17, loss : 0.0545, val_loss : 2.9387, ac

In [126]:
# 問題4
from sklearn.preprocessing import StandardScaler
import numpy as np

df = pd.read_csv("/train.csv2")
X = df.loc[:, ["GrLivArea", "YearBuilt"]]
y = df["SalePrice"]

X = np.array(X)
y = np.array(y)[:, np.newaxis]

# 標準化
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

# 対数変換
y_log = np.log(y)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_log, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)


X_train.shape, y_train.shape

((934, 2), (934, 1))

In [127]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters

    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
objective_function = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, objective_function])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    tf.random.set_random_seed(0)
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
# tf.nn.sigmoid_cross_entropy_with_logits：クロスエントロピーの値
# Y = (n, n_class)
loss_op = tf.reduce_mean(tf.square(logits - Y))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op) 
# 推定結果
#correct_pred = tf.equal(tf.sign(tf.argmax(Y, axis=1) - 1), tf.sign(tf.argmax(tf.nn.softmax(logits), axis=1) - 1))


# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            # print(mini_batch_y)
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run([loss_op], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss[0]
        total_loss /= n_samples
        val_loss = sess.run([loss_op], feed_dict={X: X_val, Y: y_val})
        #print(aq)
        #print(aq.shape)
        #print(yq)
        #print(bq)
        print("Epoch {}, loss : {}, val_loss : {}".format(epoch, total_loss, val_loss[0]))
    #test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    #print("test_acc : {:.3f}".format(test_acc))

# 考察
# 変更点は２箇所
# f.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
# tf.sign(tf.argmax(Y, axis=1) - 1), tf.sign(tf.argmax(tf.nn.softmax(logits), axis=1) - 1))
# 2値分類から多クラス分類が必要なので、多クラス分類を探した結果、ソフトマックス関数とセットのものを発見（厳密にはsigmoidではセットで作られた関数がなかった）
# 多クラスになったので、argmaxで最大のインデックスを所得して差し引くことで整合を確認



Epoch 0, loss : 153.7556497708633, val_loss : 431.3733215332031
Epoch 1, loss : 22.1956622309552, val_loss : 99.78850555419922
Epoch 2, loss : 7.104445298172422, val_loss : 45.34308624267578
Epoch 3, loss : 3.5767147122409666, val_loss : 26.44932746887207
Epoch 4, loss : 2.2332179395749177, val_loss : 18.665477752685547
Epoch 5, loss : 1.6344907493101197, val_loss : 14.6545991897583
Epoch 6, loss : 1.303751101330465, val_loss : 12.17841625213623
Epoch 7, loss : 1.0963042927247808, val_loss : 10.391828536987305
Epoch 8, loss : 0.9405969475780871, val_loss : 8.962387084960938
Epoch 9, loss : 0.8114712729699096, val_loss : 7.748530387878418
Epoch 10, loss : 0.7036697711372784, val_loss : 6.726406097412109
Epoch 11, loss : 0.614531769997558, val_loss : 5.882800579071045
Epoch 12, loss : 0.5393752776070376, val_loss : 5.183596611022949
Epoch 13, loss : 0.47532694434200673, val_loss : 4.599796772003174
Epoch 14, loss : 0.42085585859654034, val_loss : 4.105802059173584
Epoch 15, loss : 0.3746

In [128]:
# 問題5
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [129]:
# mnistデータ
train, valid, test = mnist
print(train.images.shape, valid.images.shape, test.images.shape)
print(train.labels.shape, valid.labels.shape, test.labels.shape)

#必要性はないがvalが少ないのでマージ
X = np.concatenate([train.images, valid.images], axis=0)
y = np.concatenate([train.labels, valid.labels], axis=0)

(55000, 784) (5000, 784) (10000, 784)
(55000, 10) (5000, 10) (10000, 10)


In [130]:
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
X_train.shape, y_train.shape

((38400, 784), (38400, 10))

In [134]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    tf.random.set_random_seed(0)
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
# tf.nn.sigmoid_cross_entropy_with_logits：クロスエントロピーの値
# Y = (n, n_class)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
a = tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits)
#b = tf.argmax(tf.nn.softmax(logits), axis=1)

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op) 
# 推定結果
correct_pred = tf.equal(tf.sign(tf.argmax(Y, axis=1)), tf.sign(tf.argmax(tf.nn.softmax(logits), axis=1)))


# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            # print(mini_batch_y)
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            #aq = sess.run(a, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            #bq = sess.run(b, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            #yq = sess.run(Y, feed_dict={Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        #print(aq)
        #print(aq.shape)
        #print(yq)
        #print(bq)
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

# 考察
# 変更点は２箇所
# f.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
# tf.sign(tf.argmax(Y, axis=1) - 1), tf.sign(tf.argmax(tf.nn.softmax(logits), axis=1) - 1))
# 2値分類から多クラス分類が必要なので、多クラス分類を探した結果、ソフトマックス関数とセットのものを発見（厳密にはsigmoidではセットで作られた関数がなかった）
# 多クラスになったので、argmaxで最大のインデックスを所得して差し引くことで整合を確認


Epoch 0, loss : 3.1977, val_loss : 8.3072, acc : 0.976
Epoch 1, loss : 0.5534, val_loss : 4.2393, acc : 0.981
Epoch 2, loss : 0.2849, val_loss : 2.7110, acc : 0.983
Epoch 3, loss : 0.1764, val_loss : 2.0689, acc : 0.988
Epoch 4, loss : 0.1217, val_loss : 1.7613, acc : 0.990
Epoch 5, loss : 0.0912, val_loss : 1.5421, acc : 0.990
Epoch 6, loss : 0.0717, val_loss : 1.3005, acc : 0.991
Epoch 7, loss : 0.0594, val_loss : 1.2026, acc : 0.992
Epoch 8, loss : 0.0498, val_loss : 1.0969, acc : 0.993
Epoch 9, loss : 0.0428, val_loss : 1.0321, acc : 0.993
Epoch 10, loss : 0.0373, val_loss : 0.9764, acc : 0.991
Epoch 11, loss : 0.0332, val_loss : 0.9575, acc : 0.991
Epoch 12, loss : 0.0297, val_loss : 0.9513, acc : 0.992
Epoch 13, loss : 0.0274, val_loss : 0.9316, acc : 0.992
Epoch 14, loss : 0.0243, val_loss : 0.9105, acc : 0.992
Epoch 15, loss : 0.0231, val_loss : 0.9045, acc : 0.991
Epoch 16, loss : 0.0213, val_loss : 0.8709, acc : 0.993
Epoch 17, loss : 0.0199, val_loss : 0.8724, acc : 0.994
Ep